In [1]:
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract



Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [4]:
# The website in html
audio_culture_page = read_html("https://www.audioculture.co.nz/music_index?category=Person")

# Profile names
Profiles = audio_culture_page %>% html_nodes(".skippy-col-link") %>% html_text()

# Profile links
Links = audio_culture_page %>% html_nodes(".skippy-col-link") %>% html_attr('href')

# Dataframe of profile and associated link
df = data.frame(Profiles, Links)

# Prints NZ music profiles based on chars of given string
create_profile_list = function(cafe_name) {
    # Removes special symbols and whitespace from string
    cleaned_string = gsub("([^A-Za-z0-9])+", "", cafe_name)

    # Splits string into list for iteration
    char_list = strsplit(cleaned_string, "")[[1]]

    # The list of NZ profiles based on cafe name
    profile_list = data.frame()
    
    # Iterate through every char in the given string
    for (char in char_list) {
        # Converts char into upper case for REGEX pattern
        upper_char = toupper(char)

        # Converts char into lower case for REGEX pattern
        lower_char = tolower(char)

        # Creates list of profiles that match REGEX pattern (Starts with given char)
        match_list = grep(glue('^[{upper_char}{lower_char}]'), Profiles, value = TRUE)
        
        # If the match list is not empty choose random profile from match list
        if (length(match_list) != 0) {
            # Randomly selected profile
            profile_match = sample(match_list, 1)
            
            # The profile's discography
            profile_discography = search_profile(profile_match)
            
            # Randomly selects album index from discography
            profile_album_index = sample(nrow(profile_discography), 1) 

            # Create data frame entry for profile_lsit
            profile_album = merge((data.frame(Character = upper_char, Profile = profile_match)), (profile_discography[profile_album_index,]))

        } else {
            # If no match, create empty data frame entry for profile_list
            profile_album = data.frame(Character = upper_char, Profile = NA_character_, Album = NA_character_, Featuring = NA_character_, Year = NA_character_)
        }

        # Appends profile_list with new profile
        profile_list = rbind(profile_list, profile_album)
    }
    return(profile_list)
}

# Takes a profile link as input
# Returns profile's discography with featuring artists as a dataframe
get_discography = function(name_link) {
    profile_discography_page = glue('https://www.audioculture.co.nz{name_link}/discography') %>% read_html()

    Album     = profile_discography_page %>% html_nodes("[class='header']") %>% html_text()
    Featuring = profile_discography_page %>% html_nodes("[class='body']")   %>% html_text()
    Year      = profile_discography_page %>% html_nodes("[class='year']")   %>% html_text()

    # Data frame of the given profile's discography
    discography_df = data.frame(Album, Featuring, Year)

    # Replaces empty entries with NA
    discography_df[discography_df == ""] = NA_character_

    return(discography_df)
}

# Takes a profile name as input
# Returns given name's discography (currenly no error handling if name not in dataframe)
search_profile = function(name) {
    name_link = (df %>% filter(str_detect(Profiles, name)))[[2]]

    return(get_discography(name_link))
}

In [12]:
create_profile_list('Downstairs Bar')

Character,Profile,Album,Featuring,Year
<chr>,<chr>,<chr>,<chr>,<chr>
D,Dam Native,Aotearoa Nobody Does It Better,Dam Native,2010
O,OMC,Right On,OMC,1996
W,Wahanui Wynyard,Sometimes When The Sun Shines,The Village Gossip,NA
N,Ngaire,To Sir With Dub,The Pakeha Project & Ngaire,1990
S,Scribe,Rhyme Book,Scribe (2),2007
T,Tony Hopkins,Live At The London Bar,"Phil Broadhurst, Tony Hopkins, Kevin Haines",1993
A,Arthur Baysting,Age Of Elegance,The Umbrellas,1990
I,Ike Metekingi,American Patrol / Danube Waves,The Maori Hi Five,NA
R,Rod Derrett,"Rugby, Racing And Beer","Rod Derrett, Chorus And Orchestra*",1965
